In [1]:
import networkx as nx

In [2]:
# 빈 heterogeneous graph 생성
graph = nx.MultiDiGraph()

In [3]:
# 노드 타입 정의
node_types = {
    'user': {'name': str, 'age': int, 'job': str},
    'post': {'title': str, 'content': str}
}

# 노드 추가
graph.add_node(1, node_type='user', name='Alice', age=30, job='Engineer')
graph.add_node(2, node_type='user', name='Bob', age=25, job='Designer')
graph.add_node(3, node_type='post', title='Hello', content='Welcome to the community!')
graph.add_node(4, node_type='post', title='AI', content='Artificial Intelligence is amazing!')

# 엣지 추가
graph.add_edge(1, 3, relation='authored')
graph.add_edge(2, 4, relation='authored')

# 그래프 정보 출력
print("Nodes:", graph.nodes(data=True))
print("Edges:", graph.edges(data=True))

Nodes: [(1, {'node_type': 'user', 'name': 'Alice', 'age': 30, 'job': 'Engineer'}), (2, {'node_type': 'user', 'name': 'Bob', 'age': 25, 'job': 'Designer'}), (3, {'node_type': 'post', 'title': 'Hello', 'content': 'Welcome to the community!'}), (4, {'node_type': 'post', 'title': 'AI', 'content': 'Artificial Intelligence is amazing!'})]
Edges: [(1, 3, {'relation': 'authored'}), (2, 4, {'relation': 'authored'})]


In [5]:
# 새로운 노드 정보
new_nodes = [
    {'node_id': 5, 'ntype': 'user', 'name': 'Charlie', 'age': 28, 'job': 'Data Scientist'},
    {'node_id': 6, 'ntype': 'post', 'title': 'New Post', 'content': 'This is a new post!'}
]

# 새로운 노드 추가
for node in new_nodes:
    node_id = node['node_id']
    ntype = node['ntype']
    node_data = {attr: node[attr] for attr in node_types[ntype]}
    graph.add_node(node_id, ntype=ntype, **node_data)
    
graph.add_edge(1, 6, relation='authored')
graph.add_edge(5, 4, relation='authored')

0

In [6]:
graph

In [7]:
print("Nodes:", graph.nodes(data=True))
print("Edges:", graph.edges(data=True))

Nodes: [(1, {'node_type': 'user', 'name': 'Alice', 'age': 30, 'job': 'Engineer'}), (2, {'node_type': 'user', 'name': 'Bob', 'age': 25, 'job': 'Designer'}), (3, {'node_type': 'post', 'title': 'Hello', 'content': 'Welcome to the community!'}), (4, {'node_type': 'post', 'title': 'AI', 'content': 'Artificial Intelligence is amazing!'}), (5, {'ntype': 'user', 'name': 'Charlie', 'age': 28, 'job': 'Data Scientist'}), (6, {'ntype': 'post', 'title': 'New Post', 'content': 'This is a new post!'})]
Edges: [(1, 3, {'relation': 'authored'}), (1, 6, {'relation': 'authored'}), (2, 4, {'relation': 'authored'}), (5, 4, {'relation': 'authored'})]


In [8]:
def node_add(graph, node):
    node_id = node['node_id']
    ntype = node['ntype']
    node_data = {attr: node[attr] for attr in node_types[ntype]}
    graph.add_node(node_id, ntype=ntype, **node_data)

In [9]:
def find_node_id_by_name(graph, name):
    for node_id, node_data in graph.nodes(data=True):
        if 'name' in node_data and node_data['name'] == name:
            return node_id
    return None

def edge_add(graph, user, post):
    user_id = find_node_id_by_name(graph,user)
    post_id = find_node_id_by_name(graph,post)
    
    graph.add_edge(1, 6, relation='authored')